In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device is set to", device)

# Load SQuAD 1.1 dataset

In [ ]:
from model.data import SQuAD
data = SQuAD(device=device, train_batch_size=60)
print("SQuAD 1.1 loaded")

In [ ]:
for b in iter(data.train_iter):
    batch = b
    break

# Load trained BiDAF model

In [ ]:
model_time = ""  # TODO

In [ ]:
from model.model import BiDAF
model = BiDAF(pretrain_embedding=data.WORD.vocab.vectors,
              char_vocab_size=len(data.CHAR_NESTING.vocab)).to(device)

model.load_state_dict(torch.load(f'trained_models/BiDAF_{model_time}.pt', map_location='cpu'))
model.eval()

# Pull out model forward logic to get intermediate variables

In [ ]:
# Character Embedding Layer
x_char_emb = model.char_emb(batch.x_char)
q_char_emb = model.char_emb(batch.q_char)
# Word Embedding Layer
x_word_emb = model.word_emb(batch.x_word[0])
q_word_emb = model.word_emb(batch.q_word[0])
x_lens = batch.x_word[1]
q_lens = batch.q_word[1]
x = model.highway(x_char_emb, x_word_emb)
q = model.highway(q_char_emb, q_word_emb)
# Contextual Embedding Layer
h = model.contextual_emb(x, x_lens)
u = model.contextual_emb(q, q_lens)
# Attention Flow Layer
g = self.bidaf(h, u)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.matshow(g.numpy())